# Lecture 5


## Example showing Hermite shape functions used for approximating beams 

In [15]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from plotly.io import renderers

# If LaTeX doesn't render in VS Code, try: renderers.default = "iframe_connected"
renderers.default = "notebook_connected"

# ---------------- User settings ----------------
HAT_LINE_WIDTH = 2.0      # thickness for each (unscaled) shape function (dashed)
WH_LINE_WIDTH  = 4.0      # thickness for w_h(x) (solid)
PALETTE = ["#d62728", "#2ca02c", "#ff7f0e", "#9467bd"]  # 4 colors
SHOW_NODE_LABELS = True   # annotate nodes 1..nn under the axis
YRANGE = (-1.0, 1.5)      # adjust if you widen DOF slider range

# ---------------- Mesh (1 element -> 2 nodes) ----------------
nodes = np.array([0.0, 1.0])   # single element [0, L]; change to e.g. [0.0, 2.0] for L=2
nn = len(nodes)                # 2 nodes
x1, x2 = nodes[0], nodes[1]
L = x2 - x1
x = np.linspace(x1, x2, 1000)  # dense grid on the single element

# ---------------- Hermite shape functions (Euler–Bernoulli) ----------------
def hermite_shape_functions(x, x1, x2):
    """
    Returns the four Hermite cubic shape functions on [x1, x2] evaluated on x:
      N1 -> w1,  N2 -> theta1,  N3 -> w2,  N4 -> theta2
    Conventional definitions with xi = (x - x1)/L in [0,1]:
      N1 = 1 - 3xi^2 + 2xi^3
      N2 = L*(xi - 2xi^2 + xi^3)
      N3 = 3xi^2 - 2xi^3
      N4 = L*(-xi^2 + xi^3)
    """
    L = x2 - x1
    xi = (x - x1) / L
    N1 = 1.0 - 3.0*xi**2 + 2.0*xi**3
    N2 = L * (xi - 2.0*xi**2 + xi**3)
    N3 = 3.0*xi**2 - 2.0*xi**3
    N4 = L * (-xi**2 + xi**3)
    return N1, N2, N3, N4

# Precompute the four Hermite functions on the grid
N1, N2, N3, N4 = hermite_shape_functions(x, x1, x2)
N_global = np.vstack([N1, N2, N3, N4])  # shape (4, len(x))

def assemble_w(a):
    """w_h(x) = N1*w1 + N2*theta1 + N3*w2 + N4*theta2 on the dense grid."""
    return a @ N_global

# ---------------- Initial DOFs ----------------
# a = [w1, theta1, w2, theta2]
a0 = np.array([0.5, 0.0, 1.2, 0.0], dtype=float)

# ---------------- Figure ----------------
title_txt = (
    "wₕ(x) = N₁a₁ + N₂a₂ + N₃a₃ + N₄a₄"
    # "with  ξ=x/L,  N₁=1-3ξ²+2ξ³,  N₂=L(ξ-2ξ²+ξ³),  N₃=3ξ²-2ξ³,  N₄=L(-ξ²+ξ³)"
)
fig = go.FigureWidget(layout=dict(
    title=title_txt,
    xaxis_title="x",
    yaxis_title="value",
    template="plotly_white",
    height=540
))

# 1) Unscaled Hermite functions, dashed
labels = [r"N₁(x)", r"N₂(x)", r"N₃(x)", r"N₄(x)"]
hermite_traces = []
for i, Ni in enumerate([N1, N2, N3, N4]):
    fig.add_scatter(
        x=x, y=Ni,
        mode="lines",
        name=labels[i],
        line=dict(width=HAT_LINE_WIDTH, color=PALETTE[i % len(PALETTE)], dash="solid")
    )
    hermite_traces.append(len(fig.data)-1)

# 2) w_h(x) = Σ a_i N_i(x), solid
idx_wh = len(fig.data)
fig.add_scatter(
    x=x, y=assemble_w(a0),
    mode="lines",
    # name="wₕ(x) = N₁a₁ + N₂a₂ + N₃a₃ + N₄a₄",
    name="wₕ(x)",
    line=dict(width=WH_LINE_WIDTH, color="#1f77b4", dash="solid")
)

# Optional: annotate node indices under the axis
if SHOW_NODE_LABELS:
    span = YRANGE[1] - YRANGE[0]
    y_annot = YRANGE[0] - 0.05 * span
    for j, xj in enumerate(nodes, start=1):  # 1-based
        fig.add_annotation(
            x=float(xj), y=y_annot, xref="x", yref="y",
            text=f"node {j}", showarrow=False,
            font=dict(size=11, color="#555")
        )

# Axes limits
fig.update_yaxes(range=list(YRANGE))
fig.update_xaxes(range=[float(x1), float(x2)])

# ---------------- DOF sliders (4 DOFs) ----------------
sliders = [
    widgets.FloatSlider(
        value=float(a0[i]), min=-2.0, max=2.0, step=0.1,
        description=desc, continuous_update=True, readout_format=".2f",
        layout=widgets.Layout(width='320px')
    )
    for i, desc in enumerate(["a₁", "a₂", "a₃", "a₄"])
]
controls = widgets.VBox(sliders)

def update_all(_=None):
    a = np.array([s.value for s in sliders], dtype=float)
    wh = assemble_w(a)
    with fig.batch_update():
        fig.data[idx_wh].y = wh  # only w_h updates

# Wire callbacks
for s in sliders:
    s.observe(update_all, names="value")

# Initial render
update_all()
display(fig, controls)

FigureWidget({
    'data': [{'line': {'color': '#d62728', 'dash': 'solid', 'width': 2.0},
              'mode': 'lines',
              'name': 'N₁(x)',
              'type': 'scatter',
              'uid': 'ec19cece-f894-4ee6-8676-7e1595c12ff0',
              'x': array([0.      , 0.001001, 0.002002, ..., 0.997998, 0.998999, 1.      ]),
              'y': array([1.00000000e+00, 9.99996996e-01, 9.99987992e-01, ..., 1.20079880e-05,
                          3.00400300e-06, 0.00000000e+00])},
             {'line': {'color': '#2ca02c', 'dash': 'solid', 'width': 2.0},
              'mode': 'lines',
              'name': 'N₂(x)',
              'type': 'scatter',
              'uid': '5eb877d5-bdb3-4765-9aa3-359dd0121a47',
              'x': array([0.      , 0.001001, 0.002002, ..., 0.997998, 0.998999, 1.      ]),
              'y': array([0.00000000e+00, 9.98997998e-04, 1.99399400e-03, ..., 3.99998797e-06,
                          1.00100000e-06, 0.00000000e+00])},
             {'line': {'c

## Same as above but alos plots how the rotation is approximated and the corresponding derivates of the shape functions

In [14]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display
from plotly.io import renderers

# If LaTeX doesn't render in VS Code, try: renderers.default = "iframe_connected"
renderers.default = "notebook_connected"

# ---------------- User settings ----------------
SF_LINE_WIDTH = 1.5        # unscaled shape functions, dashed
ASM_LINE_WIDTH = 4.0       # assembled curves, solid
PALETTE = ["#d62728", "#2ca02c", "#ff7f0e", "#9467bd"]  # colors for N1..N4
SHOW_NODE_LABELS = True
YRANGE_W = (-2.1, 2.1)     # y-range for deflection panel
YRANGE_TH = (-2.1, 2.1)    # y-range for rotation panel

# ---------------- Mesh (1 element -> 2 nodes) ----------------
nodes = np.array([0.0, 1.0])   # element [x1, x2]; change to e.g. [0.0, 2.0] for L=2
x1, x2 = float(nodes[0]), float(nodes[1])
L = x2 - x1
x = np.linspace(x1, x2, 1000)

# ---------------- Hermite shape functions and derivatives ----------------
def hermite_shapes_and_derivatives(x, x1, x2):
    """
    Returns N1..N4 and their x-derivatives on [x1, x2]:
      N1 -> w1, N2 -> theta1, N3 -> w2, N4 -> theta2
    """
    L = x2 - x1
    xi = (x - x1) / L
    # Shape functions
    N1 = 1.0 - 3.0*xi**2 + 2.0*xi**3
    N2 = L * (xi - 2.0*xi**2 + xi**3)
    N3 = 3.0*xi**2 - 2.0*xi**3
    N4 = L * (-xi**2 + xi**3)
    # Derivatives w.r.t. x
    dN1 = ( -6.0*xi + 6.0*xi**2 ) / L
    dN2 =   1.0 - 4.0*xi + 3.0*xi**2
    dN3 = (  6.0*xi - 6.0*xi**2 ) / L
    dN4 =  -2.0*xi + 3.0*xi**2
    return (N1, N2, N3, N4), (dN1, dN2, dN3, dN4)

# Precompute
(N1, N2, N3, N4), (dN1, dN2, dN3, dN4) = hermite_shapes_and_derivatives(x, x1, x2)
N = np.vstack([N1, N2, N3, N4])           # (4, len(x))
dN = np.vstack([dN1, dN2, dN3, dN4])      # (4, len(x))

def assemble_w(a):
    """w_h(x) = sum_i a_i * N_i(x). a = [w1, th1, w2, th2]."""
    return a @ N

def assemble_theta(a):
    """theta(x) = dw_h/dx = sum_i a_i * dN_i/dx."""
    return a @ dN

# ---------------- Initial DOFs ----------------
# a = [w1, theta1, w2, theta2]
a0 = np.array([0.5, 0.0, 1.2, 0.0], dtype=float)

# ---------------- Figure (two vertically stacked panels) ----------------
subplot_titles = (
    "Deflection  w_h(x) = N₁·w₁ + N₂·θ₁ + N₃·w₂ + N₄·θ₂",
    "Rotation  θ(x) = dw_h/dx = Σ a_i · dN_i/dx",
)
fig = make_subplots(
    rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=subplot_titles
)
fig = go.FigureWidget(fig)
fig.update_layout(template="plotly_white", height=820)

# --- Top panel: unscaled N_i (dashed) + assembled w_h (solid) ---
labels_top = [r"N₁(x) (w₁)", r"N₂(x) (θ₁)", r"N₃(x) (w₂)", r"N₄(x) (θ₂)"]
for i, Ni in enumerate([N1, N2, N3, N4]):
    fig.add_scatter(
        x=x, y=Ni, mode="lines", name=labels_top[i],
        line=dict(width=SF_LINE_WIDTH, color=PALETTE[i % len(PALETTE)], dash="solid"),
        row=1, col=1
    )
idx_wh = len(fig.data)
fig.add_scatter(
    x=x, y=assemble_w(a0), mode="lines",
    name="w_h(x)  (assembled)", line=dict(width=ASM_LINE_WIDTH, color="#1f77b4"),
    row=1, col=1
)

# --- Bottom panel: unscaled dN_i/dx (dashed) + assembled θ(x) (solid) ---
labels_bot = [r"dN₁/dx", r"dN₂/dx", r"dN₃/dx", r"dN₄/dx"]
for i, dNi in enumerate([dN1, dN2, dN3, dN4]):
    fig.add_scatter(
        x=x, y=dNi, mode="lines", name=labels_bot[i],
        line=dict(width=SF_LINE_WIDTH, color=PALETTE[i % len(PALETTE)], dash="solid"),
        row=2, col=1
    )
idx_th = len(fig.data)
fig.add_scatter(
    x=x, y=assemble_theta(a0), mode="lines",
    name="θ(x) = dw_h/dx  (assembled)", line=dict(width=ASM_LINE_WIDTH, color="#1f77b4"),
    row=2, col=1
)

# Axes formatting
fig.update_xaxes(title_text="x", range=[x1, x2], row=2, col=1)
fig.update_yaxes(title_text="w_h(x)", range=list(YRANGE_W), row=1, col=1)
fig.update_yaxes(title_text="θ(x)",   range=list(YRANGE_TH), row=2, col=1)

# Optional: annotate node indices on the bottom panel
if SHOW_NODE_LABELS:
    span = YRANGE_TH[1] - YRANGE_TH[0]
    y_annot = YRANGE_TH[0] - 0.05 * span
    for j, xj in enumerate(nodes, start=1):  # 1-based
        fig.add_annotation(
            x=float(xj), y=y_annot, xref="x", yref="y2",
            text=f"node {j}", showarrow=False,
            font=dict(size=11, color="#555")
        )

# ---------------- DOF sliders (4 DOFs) ----------------
sliders = [
    widgets.FloatSlider(
        value=float(a0[i]), min=-2.0, max=2.0, step=0.1,
        description=desc, continuous_update=True, readout_format=".2f",
        layout=widgets.Layout(width='320px')
    )
    for i, desc in enumerate(["w₁", "θ₁", "w₂", "θ₂"])
]
controls = widgets.VBox(sliders)

def update_all(_=None):
    a = np.array([s.value for s in sliders], dtype=float)
    wh = assemble_w(a)
    th = assemble_theta(a)
    with fig.batch_update():
        fig.data[idx_wh].y = wh   # assembled deflection (top panel)
        fig.data[idx_th].y = th   # assembled rotation (bottom panel)

# Wire callbacks
for s in sliders:
    s.observe(update_all, names="value")

# Initial render
update_all()
display(fig, controls)

FigureWidget({
    'data': [{'line': {'color': '#d62728', 'dash': 'solid', 'width': 1.5},
              'mode': 'lines',
              'name': 'N₁(x) (w₁)',
              'type': 'scatter',
              'uid': '190985a1-6e82-4a97-ae43-99a70a522887',
              'x': array([0.      , 0.001001, 0.002002, ..., 0.997998, 0.998999, 1.      ]),
              'xaxis': 'x',
              'y': array([1.00000000e+00, 9.99996996e-01, 9.99987992e-01, ..., 1.20079880e-05,
                          3.00400300e-06, 0.00000000e+00]),
              'yaxis': 'y'},
             {'line': {'color': '#2ca02c', 'dash': 'solid', 'width': 1.5},
              'mode': 'lines',
              'name': 'N₂(x) (θ₁)',
              'type': 'scatter',
              'uid': '166b5b27-d558-4d7d-a650-79d2c7e46412',
              'x': array([0.      , 0.001001, 0.002002, ..., 0.997998, 0.998999, 1.      ]),
              'xaxis': 'x',
              'y': array([0.00000000e+00, 9.98997998e-04, 1.99399400e-03, ..., 3.999987

## Example showing shape functions for Lagrange approximations (order 1-3) and Hermite approximation used for beams 

In [13]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display
from plotly.io import renderers

renderers.default = "notebook_connected"

# ---------------- Settings ----------------
LINE_WIDTH = 3.0
PALETTE = ["#d62728", "#2ca02c", "#ff7f0e", "#9467bd", "#8c564b"]
SHOW_NODE_LABELS = True
YRANGE = (-0.6, 1.2)  # typical range for Lagrange; Hermite N2/N4 stay moderate for L=1

# ---------------- Mesh ----------------
x1, x2 = 0.0, 1.0
L = x2 - x1
x = np.linspace(x1, x2, 1200)

# ---------------- Shape functions ----------------
def hermite_cubic_shapes(x, x1, x2):
    """
    Hermite C¹ cubic shape functions on [x1,x2].
    These are the classic Euler–Bernoulli beam interpolation functions:
        N1 = 1 - 3ξ^2 + 2ξ^3
        N2 = L(ξ - 2ξ^2 + ξ^3)
        N3 = 3ξ^2 - 2ξ^3
        N4 = L(-ξ^2 + ξ^3)
    where ξ = (x - x1)/L. Note N2, N4 carry the factor L.
    """
    L = x2 - x1
    xi = (x - x1)/L
    N1 = 1 - 3*xi**2 + 2*xi**3
    N2 = L*(xi - 2*xi**2 + xi**3)
    N3 = 3*xi**2 - 2*xi**3
    N4 = L*(-xi**2 + xi**3)
    N = np.vstack([N1, N2, N3, N4])
    labels = [r"N₁(x)", r"N₂(x)", r"N₃(x)", r"N₄(x)"]
    nodes = [x1, x2]
    return N, labels, nodes

def lagrange_basis_on_nodes(x, x_nodes):
    """
    Generic C⁰ Lagrange basis on arbitrary nodes x_nodes (1D).
    Returns N with shape (n_nodes, len(x)).
    """
    xn = np.asarray(x_nodes, dtype=float)
    n = len(xn)
    m = len(x)
    N = np.zeros((n, m))
    # denominators for each basis function
    denom = np.ones(n, dtype=float)
    for i in range(n):
        for j in range(n):
            if j != i:
                denom[i] *= (xn[i] - xn[j])
    # basis values
    for i in range(n):
        Li = np.ones(m)
        for j in range(n):
            if j != i:
                Li *= (x - xn[j])
        N[i, :] = Li / denom[i]
    return N

def lagrange_P1_shapes(x, x1, x2):
    """Linear Lagrange (P1): nodes at the endpoints."""
    nodes = [x1, x2]
    # Explicit closed-form (also could call lagrange_basis_on_nodes)
    L = x2 - x1
    N1 = (x2 - x)/L
    N2 = (x - x1)/L
    N = np.vstack([N1, N2])
    labels = [r"N₁(x)", r"N₂(x)"]
    return N, labels, nodes

def lagrange_P2_shapes(x, x1, x2):
    """Quadratic Lagrange (P2): endpoints + midpoint."""
    nodes = [x1, 0.5*(x1+x2), x2]
    N = lagrange_basis_on_nodes(x, nodes)
    labels = [r"N₁(x)", r"N₂(x)", r"N₃(x)"]
    return N, labels, nodes

def lagrange_P3_shapes(x, x1, x2):
    """Cubic Lagrange (P3): endpoints + two interior equally spaced nodes."""
    L = x2 - x1
    nodes = [x1, x1 + L/3, x1 + 2*L/3, x2]
    N = lagrange_basis_on_nodes(x, nodes)
    labels = [r"N₁(x)", r"N₂(x)", r"N₃(x)", r"N₄(x)"]
    return N, labels, nodes

# ---------------- UI ----------------
basis_dropdown = widgets.Dropdown(
    options=[
        ("Lagrange P1 (linear)", "P1"),
        ("Lagrange P2 (quadratic)", "P2"),
        ("Lagrange P3 (cubic)", "P3"),
        ("Hermite C¹ cubic", "hermite"),
    ],
    value="P1",  # start from linear
    description="Approx.:",
    layout=widgets.Layout(width='360px')
)

fig = go.FigureWidget(layout=dict(
    title="Basis functions Nᵢ(x) for u(x) approximation",
    xaxis_title="x",
    yaxis_title="Nᵢ(x)",
    template="plotly_white",
    height=540
))
fig.update_xaxes(range=[x1, x2])
fig.update_yaxes(range=list(YRANGE))

def _clear_plot():
    with fig.batch_update():
        fig.data = tuple()
        fig.layout.annotations = tuple()

def _add_node_annotations(nodes):
    if not SHOW_NODE_LABELS: 
        return
    span = YRANGE[1] - YRANGE[0]
    y_annot = YRANGE[0] + 0.05 * span
    for j, xj in enumerate(nodes, start=1):
        fig.add_annotation(
            x=float(xj), y=y_annot, xref="x", yref="y",
            text=f"node {j}", showarrow=False,
            font=dict(size=11, color="#555")
        )

def _build_for_basis(kind):
    if kind == "P1":
        N, labels, nodes = lagrange_P1_shapes(x, x1, x2)
    elif kind == "P2":
        N, labels, nodes = lagrange_P2_shapes(x, x1, x2)
    elif kind == "P3":
        N, labels, nodes = lagrange_P3_shapes(x, x1, x2)
    elif kind == "hermite":
        N, labels, nodes = hermite_cubic_shapes(x, x1, x2)
    else:
        raise ValueError("Unknown basis kind.")

    _clear_plot()
    for i in range(N.shape[0]):
        fig.add_scatter(
            x=x, y=N[i], mode="lines",
            name=labels[i],
            line=dict(width=LINE_WIDTH, color=PALETTE[i % len(PALETTE)], dash="solid")
        )
    _add_node_annotations(nodes)

def _on_basis_change(change):
    if change["name"] == "value":
        _build_for_basis(change["new"])

basis_dropdown.observe(_on_basis_change, names="value")
_build_for_basis(basis_dropdown.value)
display(widgets.HBox([basis_dropdown]), fig)

FigureWidget({
    'data': [{'line': {'color': '#d62728', 'dash': 'solid', 'width': 3.0},
              'mode': 'lines',
              'name': 'N₁(x)',
              'type': 'scatter',
              'uid': '88e0d15d-7a69-415a-8cd6-133cd35b358e',
              'x': array([0.00000000e+00, 8.34028357e-04, 1.66805671e-03, ..., 9.98331943e-01,
                          9.99165972e-01, 1.00000000e+00]),
              'y': array([1.00000000e+00, 9.99165972e-01, 9.98331943e-01, ..., 1.66805671e-03,
                          8.34028357e-04, 0.00000000e+00])},
             {'line': {'color': '#2ca02c', 'dash': 'solid', 'width': 3.0},
              'mode': 'lines',
              'name': 'N₂(x)',
              'type': 'scatter',
              'uid': 'fd5765ad-d379-480c-ad4f-2775cfb405a6',
              'x': array([0.00000000e+00, 8.34028357e-04, 1.66805671e-03, ..., 9.98331943e-01,
                          9.99165972e-01, 1.00000000e+00]),
              'y': array([0.00000000e+00, 8.34028357e-04,